## Chargement du dataset

In [1]:
import csv
import pandas as pd

file_path_train = "./data/train_submission.csv"
file_path_test = "./data/test_without_labels.csv"

data_train = pd.read_csv(file_path_train)


## Analyse du dataset d'entraînement

### On observe d'abord les données non labellisées

In [3]:
data_train_without_label = data_train[data_train["Label"].isna()]

In [4]:
data_train_without_label

,Usage,Text,Label
107,Public,Kòe bô jōa kú hō͘-sū sió-chiá lâi kā góan mn̄...,NaN
803,Public,Söğütçük sī chi̍t ê tī Türkiye Aydın séng Çine...,NaN
1095,Public,Golden Valley Kūn ū khó-lêng sī kóng:,NaN
1894,Public,Tī Montégut-Lauragais ê sì-ûi ū Nogaret Revel...,NaN
2499,Public,Soveria Simeri ùi séng lāi ê hoān-ûi.,NaN
...,...,...,...
189637,Public,Bellebrune sī ūi-tī Hoat-kok Nord-Pas-de-Calai...,NaN
189946,Public,Bô phah-sǹg tī sin-le̍k 10 go̍eh 29 hō ē-po͘ ...,NaN
189959,Public,Wiejki sī chi̍t ê tī Pho-lân Kiōng-hô-kok Podl...,NaN
190397,Public,Tī pún só͘-chāi sì-ûi ê tē-hng ū Valy Veselí ...,NaN


Il y a 500 instances qui ne sont pas labellisées. 

On choisit de se débarasser de ces données pour l'entraînement de nos modèles.

In [5]:
data_train_without_nan_for_label = data_train.dropna() # suppression des données non labellisées

In [6]:
number_of_languages = len(data_train["Label"].unique())
print(f"Il y a {number_of_languages} différentes langues dans le dataset de train")

Il y a 390 différentes langues dans le dataset de train


### Analyse des données labellisées

In [7]:
dataset_sorted_by_number_instances_by_language = data_train_without_nan_for_label.groupby("Label").count().sort_values('Usage', ascending=False)
dataset_sorted_by_number_instances_by_language

,Usage,Text
Label,,
tgk,1500,1500
guj,1000,1000
tat,1000,1000
crh,1000,1000
kaa,1000,1000
...,...,...
gil,2,2
toi,1,1
gaa,1,1


On observer que le nombre d'exemples par langue varie. Certaines langues sont sur-représentées (avec 1500 instances pour la première) par rapport à d'autres. 

In [8]:
percentage_of_languages_with_at_least_100_instances = len(dataset_sorted_by_number_instances_by_language[dataset_sorted_by_number_instances_by_language["Usage"] >= 100])/len(dataset_sorted_by_number_instances_by_language) * 100
print(f"Le pourcentage de langues avec au moins 100 instances est {percentage_of_languages_with_at_least_100_instances}%")

Le pourcentage de langues avec au moins 100 instances est 93.31619537275064%


## Pré-traitement du dataset de train

In [99]:
import string
import re 
import unicodedata

def cleaning(text): 
    """
    Fonction pour pré-traiter le texte en enlevant tous les éléments de ponctuation, les chiffres, les double espaces, les URL etc. 
    """

    if not isinstance(text, str):
        return ""

    text = re.sub(r"\s+", " ", text).strip() # enlève les double espaces
    # text = re.sub(r"\(.*?\)|\[.*?\]|\{.*?\}|['\"«»„“”‘’]|\<.*?\>", " ", text) # 1. Supprimer les textes entre (), [], {}, "", « »
    text = re.sub(r"https?://[^\s]+|www\.[^\s]+", " ", text) # Supprimer les URLs
    text = re.sub(r"\b[A-Z]+\d*[A-Z\d]*", " ", text) # Supprimer les sigles type "IK10", "ABC123", "X4D" (au moins 1 lettre + au moins 1 chiffre)
    text = re.sub(r"\b[A-ZÀ-ÖØ-Þ][a-zà-öø-ÿ]*", " ", text) # Supprimer les mots qui commencent par une majuscule (prénoms, noms propres, etc.)
    text = re.sub(r"\d+", " ", text)  # Supprimer les nombres isolés
    text = text.translate(str.maketrans("", "", string.punctuation))  # Supprimer la ponctuation et les caractères spéciaux
    text = ''.join(c for c in text if unicodedata.category(c)[0] not in ["C", "S"])  # Supprimer les caractères de contrôle Unicode, symboles et emojis

    asian_punctuation = "，。？！《》【】（）；：、。" # Liste de ponctuation à inclure pour les langues asiatiques
    text = text.replace('-', ' ') # enlève les tirets 
    text = text.translate(str.maketrans('', '', string.punctuation + asian_punctuation)) # supprime la ponctuation asiatique
    
    text_cleaned = text.lower() # met le texte en minuscule 

    return(text_cleaned)


On choisit de ne pas enlever le texte entre (), [], {}, "", « » car on s'aperçoit que les résultats sont moins bons, cela nous fait perdre de l'information utile. 

### Création d'un ensemble de mots anglais pour pouvoir enlever les mots anglais dans les phrases avec des mots anglais mélangés à d'autres langues

In [11]:
import nltk
from nltk.corpus import words

# Télécharger la liste des mots en anglais (une seule fois nécessaire)
nltk.download('words')

# Liste des mots en anglais
english_words = set(word.lower() for word in words.words())


[nltk_data] Downloading package words to
[nltk_data]     /Users/hippolytelecomte/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
data_ang = data_train_without_nan_for_label[data_train_without_nan_for_label["Label"] == 'eng']["Text"]

# Collecte des mots uniques
for text in data_ang:
    for word in text.split():
        english_words.add(word.lower())



In [13]:
def remove_most_english_words(text): 
    """
    Fonction pour enlever les mots anglais lorsque la langue du texte n'est pas l'anglais. 
    """
    tokens = text.split() 
    filtered_tokens = [word for word in tokens if word.lower() not in english_words]

    return ' '.join(filtered_tokens)

Finalement on s'aperçoit qu'enlever les mots anglais dans les phrases qui ne sont pas labellisées comme du texte anglais baisse nos résultats. 

## Première approche sans tokenizer avec TFIDF et MultinomialNB

Séparation entre le train et le val

In [14]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(data_train_without_nan_for_label, test_size=0.2, random_state=42)

Application du pré-traitement sur tout le dataframe

In [100]:
from tqdm import tqdm
tqdm.pandas()  

def pre_processing(df, remove_espace = True, not_test = True, need_to_clean = True): 
    """
    Utilisation des méthodes de pré-traitement définies auparavant pour rendre le texte propre. 
    """

    if need_to_clean: 
        df['Text'] = df['Text'].apply(cleaning)
    
    if not_test: 
        df['Text'] = df.progress_apply(
            lambda row: remove_most_english_words(row['Text']) if row['Label'] != 'eng' else row['Text'], axis=1
        )
    
    if remove_espace: 
        df['Text'] = df['Text'].str.replace(' ', '', regex=False)
    
    return df


On s'apperçoit également que dans le dataset d'entraînement il y a des langues qui peuvent écrites dans différents alphabets, c'est le cas pour certaines langues asiatiques notamment. On choisit ainsi de créer une fonction qui permet de détecter les alphabets présents dans la phrase et de déterminer l'alphabet majoritaire en fonction du nombre de caractères (même si cette méthode n'est pas optimale car certains alphabets contiennent plus d'informations dans leur caractère que d'autres). Ensuite on change le label des phrases en ajoutant l'alphabet utilisé pour la langue et créant ainsi de nouvelles catégories comme fra_Latin pour le français écrit avec l'alphabet latin. 

In [16]:
import unicodedata
from collections import defaultdict

# Catégorisation fine des scripts
SCRIPT_MAP = {
    "LATIN": "Latin",
    "CYRILLIC": "Cyrillique",
    "ARABIC": "Arabe",
    "HEBREW": "Hébreu",
    "GREEK": "Grec",
    "DEVANAGARI": "Devanagari (Hindi, Sanskrit)",
    "HIRAGANA": "Hiragana (Japonais)",
    "KATAKANA": "Katakana (Japonais)",
    "CJK": "Kanji (Chinois, Japonais, Coréen)",
    "HANGUL": "Hangul (Coréen)",
    "THAI": "Thaï",
    "ARMENIAN": "Arménien",
    "GEORGIAN": "Géorgien",
    "ETHIOPIC": "Éthiopien",
    "TAMIL": "Tamoul",
    "BENGALI": "Bengali",
    "TELUGU": "Télougou",
}

def count_alphabet_characters(text):
    script_counts = defaultdict(int)

    for char in text:
        if char.isalpha():  # On ignore les symboles et ponctuations
            try:
                char_name = unicodedata.name(char) 
                script_key = char_name.split()[0]  # Prend le premier mot du nom Unicode
                
                if "CJK" in char_name:
                    script_key = "CJK"  # Les kanji sont classés sous "CJK UNIFIED IDEOGRAPH"
                
                script_name = SCRIPT_MAP.get(script_key, script_key)  # Utilise le mapping o
                script_counts[script_name] += 1  # Incrémente le compteur
                
            except ValueError:
                continue  # Si le caractère n'a pas de nom Unicode
    
    return dict(script_counts)  # Retourne un dictionnaire des comptages

def most_frequent_script(text):
    script_counts = count_alphabet_characters(text)  # Appel de la fonction précédente
    
    if script_counts:  # Vérifie si le dictionnaire n'est pas vide
        most_common_script = max(script_counts.items(), key=lambda x: x[1])  # Trouve l'alphabet avec le max de caractères
        return most_common_script  # Retourne (nom de l'alphabet, nombre d'occurrences)
    else:
        return None  # Retourne None si aucun alphabet trouvé

from tqdm import tqdm

def add_alphabet_to_label(df):
    for index, row in tqdm(df.iterrows(), total=len(df)):  # Parcourt chaque ligne du DataFrame
        alphabet_most_frequent = most_frequent_script(row['Text'])  # Détecte l'alphabet dominant
        
        if alphabet_most_frequent:  # Vérifie si un alphabet a été trouvé
            df.at[index, 'Label'] = f"{row['Label']}_{alphabet_most_frequent[0]}"  # Met à jour le label
    
    return df


Application du pré-traitement et du changement de labellisation sur le train et le val. 

In [ ]:
train_set_first_version = train_set.copy()
val_set_first_version = val_set.copy()
train_set_first_version = pre_processing(train_set_first_version, remove_espace=False, not_test=False) 
val_set_first_version = pre_processing(val_set_first_version, remove_espace=False, not_test=False)
train_set_first_version = add_alphabet_to_label(train_set_first_version)
val_set_first_version = add_alphabet_to_label(val_set_first_version)

100%|██████████| 38020/38020 [00:03<00:00, 11916.76it/s]


Mise en place de la pipeline pour le modèle où on choisit d'utiliser comme Vectorizer TFIDF et comme modèle MultinomialNB.

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(1, 4), max_features=200000)
x_train = train_set_first_version['Text'].tolist()
y_train = train_set_first_version['Label'].tolist()
x_val = val_set_first_version['Text'].tolist()
y_val = val_set_first_version['Label'].tolist()
y_total = y_train + y_val

# converting categorical variables to numerical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_total)

y_train = le.transform(y_train)
y_val = le.transform(y_val)
label_mapping = dict(zip(le.classes_, range(len(le.classes_))))


x_train = vectorizer.fit_transform(x_train)
x_val = vectorizer.transform(x_val)

In [103]:
naive_bayes = MultinomialNB(alpha= 0.001, fit_prior = False) 
naive_bayes.fit(x_train,y_train)

MultinomialNB(alpha=0.001, fit_prior=False)

In [104]:
from sklearn.metrics import accuracy_score, classification_report

predictions = naive_bayes.predict(x_val)
accuracy = accuracy_score(y_val, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8497369805365597


Création de fonctions pour récupéer les labels originaux. 

In [105]:
predicted_labels = le.inverse_transform(predictions)
labels_predict = le.inverse_transform(y_val)

import numpy as np 

def restore_original_label(label):
    return label.split("_")[0]  # Prend seulement la première partie avant '_'

def restore_labels(liste):
    new_liste = []
    for element in tqdm(liste): 
        new_liste.append(restore_original_label(element))
    return np.array(new_liste)

In [106]:
final_pred = restore_labels(predicted_labels)
val_predict = restore_labels(labels_predict)
final_accuracy = accuracy_score(val_predict, final_pred)
print("Accuracy:", final_accuracy)


100%|██████████| 38020/38020 [00:00<00:00, 1489528.56it/s]

Accuracy: 0.8514992109416096


### Analyse des résultats (sûrement à améliorer)

In [115]:
import numpy as np

# Obtenir les indices des classes présentes dans y_val
present_classes = np.unique(np.concatenate((y_val, predictions)))

# Extraire uniquement les noms correspondants
filtered_target_names = [le.classes_[i] for i in present_classes]

# Générer le rapport de classification sous forme de dictionnaire
report = classification_report(y_val, predictions, target_names=filtered_target_names, output_dict=True)

# Filtrer les classes (en excluant 'accuracy', 'macro avg', 'weighted avg')
filtered_report = {label: metrics for label, metrics in report.items() if isinstance(metrics, dict)}

# Trier les langues par F1-score de manière décroissante
sorted_report = sorted(filtered_report.items(), key=lambda x: x[1]['f1-score'], reverse=True)

# Afficher le rapport trié
print("Classification Report (trié par F1-score décroissant):\n")
for label, metrics in sorted_report:
    print(f"{label}: F1-score = {metrics['f1-score']:.4f}, Precision = {metrics['precision']:.4f}, Recall = {metrics['recall']:.4f}, Support = {metrics['support']}")


Classification Report (trié par F1-score décroissant):

abk_Cyrillique: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 116.0
ada_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 1.0
ahk_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 89.0
alt_Cyrillique: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 90.0
aoj_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 98.0
arn_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 115.0
asm_Bengali: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 99.0
bpy_Bengali: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 95.0
bzj_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 104.0
cab_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 83.0
cak_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 103.0
chk_Latin: F1-score = 1.0

/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sample

In [52]:
pd.set_option('display.max_rows', 500)

In [53]:
val_set_first_version[val_set_first_version['Label'] == "tgk_Arabe"]

,Usage,Text,Label
85817,Public,وزیراعظم پیروں کی اولاد ہیں لینے پر آئیں تو در...,tgk_Arabe
80221,Public,یہ آئین و قانون اور متاثرہ عوام کے درمیان بڑے...,tgk_Arabe
137438,Public,لَوْ لاَ الشُّيُوخُ الرُكَّعُ وَاْلبَهائِمُ ال...,tgk_Arabe
26667,Public,فقر اوہ جنہاں فکر ن کوئی، جیڑھے رب دے راہ وکانے,tgk_Arabe
50294,Public,قال سلمان وإن هذا لكائن يا رسول الله؟ قال صلى ...,tgk_Arabe
189041,Public,جی پی ایس موجود نہیں، براہِ کرم اپنے موبائل کی...,tgk_Arabe
81019,Public,پھر اگر یہ بھی فرض کر لیا جائے کہ اس نے طاقت ک...,tgk_Arabe
78097,Public,آج کے افسانہ نگارپرکیچڑ اچھالنے والے حقیقتِ ح...,tgk_Arabe
32309,Public,ملک پورملک مشتاق کے گھر ایک عظیم الشان محفل من...,tgk_Arabe
63494,Public,۔ قیمت کا تعین بھی کرتے ہیں کیونکہ اس سے کتاب ...,tgk_Arabe


## Deuxième approche avec SentencePiece comme tokenizer

### Génération d'un fichier brut .txt pour entraîner SentencePiece

In [38]:
# Extraire uniquement la colonne "Text"
corpus_path = "corpus_multilingue.txt"  # Chemin de sortie pour le corpus
data_train_preprocessed_for_corpus = data_train.copy()
data_train_preprocessed_for_corpus = pre_processing(data_train_preprocessed_for_corpus, remove_espace=False, not_test=True, need_to_clean=True)
data_train_preprocessed_for_corpus["Text"].dropna().to_csv(corpus_path, index=False, header=False, sep="\n")

print(f"Corpus enregistré : {corpus_path}, avec {len(data_train)} phrases.")

100%|██████████| 190599/190599 [00:02<00:00, 82645.12it/s]


Corpus enregistré : corpus_multilingue.txt, avec 190599 phrases.


### Entraînement de SentencePiece et chargement du modèle

In [39]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input='./data/corpus_multilingue.txt',  
    model_prefix='sp_model',
    vocab_size=60000,  
    character_coverage=1.0,  
    model_type='unigram'  
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./data/corpus_multilingue.txt
  input_format: 
  model_prefix: sp_model
  model_type: UNIGRAM
  vocab_size: 60000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_priva

In [40]:
sp = spm.SentencePieceProcessor(model_file='sp_model.model')

def sentencepiece_tokenize(text):
    """Tokenise un texte en sous-mots avec SentencePiece"""
    return ' '.join(sp.encode(text, out_type=str))

On entraîne le tokenizer sur du texte pré-traité en revanche on l'infère sur du texte brut car il est capable de le gérer directement.

In [57]:
train_set_second_version = train_set.copy()
val_set_second_version = val_set.copy()
# train_set_second_version = pre_processing(train_set_second_version, remove_espace=False, not_test=True, need_to_clean=False)
# val_set_second_version = pre_processing(val_set_second_version, remove_espace=False, not_test=True, need_to_clean=False)

In [58]:
train_set_second_version = add_alphabet_to_label(train_set_second_version)
val_set_second_version = add_alphabet_to_label(val_set_second_version)

100%|██████████| 38020/38020 [00:03<00:00, 12041.81it/s]


In [59]:
# Appliquer SentencePiece 
train_set_second_version['Text'] = train_set_second_version['Text'].progress_apply(sentencepiece_tokenize)
val_set_second_version['Text'] = val_set_second_version['Text'].progress_apply(sentencepiece_tokenize)


100%|██████████| 38020/38020 [00:01<00:00, 20856.18it/s]


Mise en place de la même pipeline que précédemment, on a juste tokenizé en amont les phrases.

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

vectorizer_sp = TfidfVectorizer(analyzer="char", ngram_range=(1, 4), max_features=200000)
naive_bayes_sp = MultinomialNB(alpha= 0.001, fit_prior = False) 

pipeline = Pipeline([
    ('tfidf', vectorizer_sp),
    ('mnb', naive_bayes_sp)
])

x_train_sp = train_set_second_version['Text'].tolist()
y_train_sp = train_set_second_version['Label'].tolist()
x_val_sp = val_set_second_version['Text'].tolist()
y_val_sp = val_set_second_version['Label'].tolist()
y_total_sp = y_train_sp + y_val_sp

# converting categorical variables to numerical
from sklearn.preprocessing import LabelEncoder
le_sp = LabelEncoder()
le_sp.fit(y_total_sp)

y_train_sp = le_sp.transform(y_train_sp)
y_val_sp = le_sp.transform(y_val_sp)
label_mapping = dict(zip(le_sp.classes_, range(len(le_sp.classes_))))


pipeline.fit(x_train_sp, y_train_sp)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer='char', max_features=200000,
                                 ngram_range=(1, 4))),
                ('mnb', MultinomialNB(alpha=0.001, fit_prior=False))])

In [45]:
from sklearn.metrics import accuracy_score, classification_report

predictions_sp = pipeline.predict(x_val_sp)
accuracy_sp = accuracy_score(y_val_sp, predictions_sp)
print("Accuracy:", accuracy_sp)

Accuracy: 0.8505523408732246


Récupération des labels originaux

In [46]:
predicted_labels_sp = le_sp.inverse_transform(predictions_sp)
labels_to_predict = le_sp.inverse_transform(y_val_sp)

In [47]:
import numpy as np 

def restore_original_label(label):
    return label.split("_")[0]  # Prend seulement la première partie avant '_'

def restore_labels(liste):
    new_liste = []
    for element in tqdm(liste): 
        new_liste.append(restore_original_label(element))
    return np.array(new_liste)


In [48]:
final_prediction = restore_labels(predicted_labels_sp)
val_to_predict = restore_labels(labels_to_predict)
final_accuracy = accuracy_score(val_to_predict, final_prediction)
print("Accuracy:", final_accuracy)


100%|██████████| 38020/38020 [00:00<00:00, 1511482.39it/s]

Accuracy: 0.8524723829563388


### Analyse des résultats

In [49]:
present_classes_sp = np.unique(np.concatenate((y_val_sp, predictions_sp)))

# Extraire uniquement les noms correspondants
filtered_target_names_sp = [le_sp.classes_[i] for i in present_classes_sp]

In [50]:
# Générer le rapport de classification sous forme de dictionnaire
report_sp = classification_report(y_val_sp, predictions_sp, target_names=filtered_target_names_sp, output_dict=True)

# Filtrer les classes (en excluant 'accuracy', 'macro avg', 'weighted avg')
filtered_report = {label: metrics for label, metrics in report_sp.items() if isinstance(metrics, dict)}

# Trier les langues par F1-score de manière décroissante
sorted_report = sorted(filtered_report.items(), key=lambda x: x[1]['f1-score'], reverse=True)

# Afficher le rapport trié
print("Classification Report (trié par F1-score décroissant):\n")
for label, metrics in sorted_report:
    print(f"{label}: F1-score = {metrics['f1-score']:.4f}, Precision = {metrics['precision']:.4f}, Recall = {metrics['recall']:.4f}, Support = {metrics['support']}")


Classification Report (trié par F1-score décroissant):

abk_Cyrillique: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 116.0
ach_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 22.0
ada_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 1.0
ahk_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 89.0
alt_Cyrillique: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 90.0
aoj_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 98.0
arn_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 115.0
asm_Bengali: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 99.0
bpy_Bengali: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 95.0
bzj_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 104.0
cab_Latin: F1-score = 1.0000, Precision = 1.0000, Recall = 1.0000, Support = 83.0
cak_Latin: F1-score = 1.00

/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sample

In [166]:
val_set_second_version[val_set_second_version['Label'] == 'bos_Latin']

,Usage,Text,Label
34789,Public,▁ B . ▁ 2 4 SI ▁ - ▁ I stra ži vanje ▁od sje k...,bos_Latin
187584,Public,▁ I ▁svak og ▁da na ▁nakon ▁to ga ▁je ▁laga no...,bos_Latin
172443,Public,▁ S lu ša j ! ▁ G de ▁je ▁čo ve k ▁koji ▁je ▁p...,bos_Latin
1390,Public,▁ E ki pa ▁ J a pan ske ▁se ▁sas toj ala ▁od ▁...,bos_Latin
49647,Public,▁ Ž ene ▁će ▁pu ca ti ▁ako ▁su ▁iz gu bile ▁si...,bos_Latin
...,...,...,...
146261,Public,▁ N o ▁proizvod it ▁će ▁ih ▁i ▁sam ▁jedno m ▁k...,bos_Latin
170926,Public,"▁ "" C rk va ▁ Š klop ot nica ▁ ( c rk va ▁sv ....",bos_Latin
152952,Public,▁ T ako đe ▁nekoliko ▁ta ča ka ▁je ▁ski nu to ...,bos_Latin
156427,Public,▁ S vi ▁koji ▁su ▁radi li ▁sa ▁tvoj im ▁oc em ...,bos_Latin


## Utilisation du meilleur modèle pour le test set 

Entraînement du meilleur modèle sur tout le train set

In [110]:
train = data_train_without_nan_for_label.copy()
train = pre_processing(train, remove_espace=False, not_test=False)
train = add_alphabet_to_label(train)
# train['Text'] = train['Text'].progress_apply(sentencepiece_tokenize)
x = train['Text'].tolist()
y = train['Label'].tolist()

vectorizer= TfidfVectorizer(analyzer="char", ngram_range=(1, 4), max_features=200000)
naive_bayes = MultinomialNB(alpha= 0.001, fit_prior = False) 

best_pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('mnb', naive_bayes)
])

from sklearn.preprocessing import LabelEncoder
le_test = LabelEncoder()
y = le_test.fit_transform(y)
label_mapping_test = dict(zip(le_test.classes_, range(len(le_test.classes_))))

best_pipeline.fit(x, y)

100%|██████████| 190099/190099 [00:15<00:00, 12378.64it/s]


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer='char', max_features=200000,
                                 ngram_range=(1, 4))),
                ('mnb', MultinomialNB(alpha=0.001, fit_prior=False))])

Prédiction des labels pour le test et génération du csv à déposer

In [ ]:
data_test= pd.read_csv(file_path_test)
test_set = pre_processing(data_test, remove_espace=False, not_test=False)
# test_set['Text'] = test_set['Text'].progress_apply(sentencepiece_tokenize)

x_test = test_set['Text'].tolist()
predictions_test = best_pipeline.predict(x_test)

In [112]:
predicted_labels_test = le_test.inverse_transform(predictions_test)
predicted_labels_test = restore_labels(predicted_labels_test)
test_set['Label'] = predicted_labels_test

100%|██████████| 190567/190567 [00:00<00:00, 1302890.45it/s]


In [113]:
column_ID = [i for i in range(1, len(test_set)+1)]
test_set['ID'] = column_ID

In [114]:
test_set[['Label', 'ID']].to_csv('test_set_v7_sans_tokenizer_predicted.csv', index=False)